In [34]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json

In [35]:
credentials = json.load(open('credentials.json'))
CLIENT_ID = credentials['client_id']
CLIENT_SECRET = credentials['client_secret']

In [36]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [62]:
playlists = json.load(open('playlists.json'))
user_list = []
id_list = []
for playlist_index in range(len(playlists)):
    user_list.append(playlists[playlist_index]['uri'].split(':')[2])
    id_list.append(playlists[playlist_index]['uri'].split(':')[4])

In [63]:
print(id_list)

['5SesJdY6HgfoCrxpEO8qaA']


In [57]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [58]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "artist_id", "album", "track_name", "track_id", "release_date",
                             "danceability", "energy", "key", "loudness", "mode", "speechiness","acousticness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = get_playlist_tracks(creator, playlist_id)
    for track in playlist:
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["artist_id"] = track["track"]["album"]["artists"][0]["id"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["release_date"] =  track["track"]["album"]["release_date"]
        
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [64]:
for index in range(len(playlists)):
    playlist_df = analyze_playlist(user_list[index], id_list[index])

In [65]:
playlist_df.to_csv('file1.csv',index = False)